<a href="https://colab.research.google.com/github/niranjan-sa/DataScience/blob/master/Innoplexus_LSTM_Sentiment_Analysis_Final%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.listdir('drive/')
train = pd.read_csv('drive/My Drive/Data Science Preparation/Innoplexus Sentiment Analysis/train_F3WbcTw.csv')
test = pd.read_csv('drive/My Drive/Data Science Preparation/Innoplexus Sentiment Analysis/test_tOlRoBf.csv')

In [4]:
train.head(3)

,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2
2,fe809672251f6bd0d986e00380f48d047c7e7b76,Interesting that it only targets S1P-1/5 recep...,fingolimod,2


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


Using TensorFlow backend.


In [0]:
data = train.append(test, ignore_index=True, sort=False)

In [7]:
data.head(2)

,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2.0
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2.0


In [8]:
data.tail(2)

,unique_hash,text,drug,sentiment
8201,9936efcb83eded79fe9454df188edd7b96c6109e,It sounds like you're doing very well FG! I wo...,stelara,NaN
8202,05402df12d6769d7f38ab40e0b81464e65e1df0a,"Hi @rambles too! It’s such a hard decision, is...",cladribine,NaN


In [0]:
data = data[['text', 'sentiment']]

In [10]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 0].size)
print(data[ data['sentiment'] == 1].size)
print(data[ data['sentiment'] == 2].size)

1234
1674
7650


In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [12]:
embed_dim = 128//4
lstm_out = 196//4

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['categorical_accuracy'])
print(model.summary())

W0728 05:21:01.909246 140480444086144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0728 05:21:01.967686 140480444086144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0728 05:21:01.977338 140480444086144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0728 05:21:02.008285 140480444086144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0728 05:21:02.020948 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 24184, 32)         64000     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 24184, 32)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 49)                16072     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 150       
Total params: 80,222
Trainable params: 80,222
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
train = X[:5279,:]
test = X[5279:, :]

In [0]:
ytrain = data['sentiment'].iloc[:5279]

In [15]:
Y = pd.get_dummies(ytrain).values
X_train, X_test, Y_train, Y_test = train_test_split(train,Y, test_size = 0.3, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(3695, 24184) (3695, 3)
(1584, 24184) (1584, 3)


In [16]:
import gc
del train, X
gc.collect()

24

In [17]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 1)

W0728 05:21:03.588018 140480444086144 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
3695/3695 [==============================] - 2192s 593ms/step - loss: 0.8276 - categorical_accuracy: 0.7221
Epoch 2/10
3695/3695 [==============================] - 2176s 589ms/step - loss: 0.7602 - categorical_accuracy: 0.7261
Epoch 3/10
3695/3695 [==============================] - 2165s 586ms/step - loss: 0.7134 - categorical_accuracy: 0.7264
Epoch 4/10
3695/3695 [==============================] - 2166s 586ms/step - loss: 0.6723 - categorical_accuracy: 0.7378
Epoch 5/10
3695/3695 [==============================] - 2170s 587ms/step - loss: 0.6335 - categorical_accuracy: 0.7464
Epoch 6/10
3695/3695 [==============================] - 2170s 587ms/step - loss: 0.6119 - categorical_accuracy: 0.7583
Epoch 7/10
3695/3695 [==============================] - 2172s 588ms/step - loss: 0.5872 - categorical_accuracy: 0.7754
Epoch 8/10
3695/3695 [==============================] - 2175s 589ms/step - loss: 0.5696 - categorical_accuracy: 0.7767
Epoch 9/10
3695/3695 [==========================

In [0]:
del X_train, Y_train

In [19]:
import gc
gc.collect()

90

90

In [20]:
preds = model.predict_classes(X_test, verbose=1)
pd.Series(preds).value_counts()

1584/1584 [==============================] - 138s 87ms/step


2    1359
0     114
1     111
dtype: int64

2    1359
0     114
1     111
dtype: int64

In [21]:
final = model.predict_classes(test, verbose=1)
pd.Series(preds).value_counts()

2924/2924 [==============================] - 253s 86ms/step


2    1359
0     114
1     111
dtype: int64

2    1359
0     114
1     111
dtype: int64

In [27]:
pd.Series(final).value_counts()

2    2582
1     171
0     171
dtype: int64

In [31]:
preds

array([2, 2, 2, ..., 2, 2, 2])

In [33]:
Y_test

array([[0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]], dtype=uint8)

In [34]:
from sklearn.metrics import accuracy_score
print (accuracy_score(np.argmax(Y_test,axis=1), preds))

0.6679292929292929


In [22]:
final

array([2, 1, 2, ..., 2, 2, 2])

array([2, 1, 2, ..., 2, 2, 2])

In [0]:
test = pd.read_csv('drive/My Drive/Data Science Preparation/Innoplexus Sentiment Analysis/sample_submission_i5xnIZD.csv')
test.head()
test['sentiment'] = final
test.to_csv('submission_LSTM.csv', index=False)

In [0]:
model.save_weights('drive/My Drive/Data Science Preparation/Innoplexus Sentiment Analysis/best_models.h5')

In [0]:
model.save_weights('drive/My Drive/Data Science Preparation/Innoplexus Sentiment Analysis/best_models_one.h5')

In [0]:
test.to_csv('drive/My Drive/Data Science Preparation/Innoplexus Sentiment Analysis/submission_LSTM_two.csv', index=False)